# OER Volcano for IrOx systems

***

# Import Modules

In [1]:
%%capture
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

sys.path.insert(
    0,
    os.path.join(
        os.environ["PROJ_irox"],
        "workflow",
        ),
    )

sys.path.insert(
    0,
    os.path.join(
        os.environ["PROJ_irox"],
        "data",
        ),
    )

from an_data_processing import load_df

###########################################################

# Python Modules
import numpy as np
import pandas as pd

import plotly.plotly as py
import plotly.graph_objs as go

# import colorlover as cl
from IPython.display import HTML

# My Modules
from orr_reaction.orr_fed_plot import ORR_Free_E_Plot
from orr_reaction.orr_fed_plot import Scaling_Relations_Plot
from orr_reaction.orr_fed_plot import Volcano_Plot

# Project Data
from proj_data_irox import (
    surface_energies,
    smart_format_dict,
    smart_format_dict_volcano,

    irox_bulk_color_map,
    color_palettes,
    system_color_map,

    max_surf_e,
    min_surf_e,
    proj_dir_name,
    smart_format_dict,
    
    gas_molec_dict,
    scaling_dict_ideal,
    exp_irox_lim_pot,
    data_dir,
    
    groupby_props,
    )

Divisor is Energy class instance!!!
Divisor is Energy class instance!!!
TEMP TEMP TEMPTEMP TEMP TEMPTEMP TEMP TEMPTEMP TEMP TEMPTEMP TEMP TEMPTEMP TEMP TEMPTEMP TEMP TEMPTEMP TEMP TEMPTEMP TEMP TEMPTEMP TEMP TEMPTEMP TEMP TEMPTEMP TEMP TEMPTEMP TEMP TEMPTEMP TEMP TEMPTEMP TEMP TEMPTEMP TEMP TEMPTEMP TEMP TEMPTEMP TEMP TEMPTEMP TEMP TEMPTEMP TEMP TEMP
-7.45942759
-3.38574595


# Script Inputs

In [3]:
save_plot = True
plot_exp_traces = True

plot_range = {
    "y": [2., 1.4],
    "x": [1., 2.],
    }

# Read and Process Data Frame

## Read dataframe from file

In [4]:
%%capture

df_pourbaix, df_ads, df_surf = load_df(
    from_file=False,
    root_dir=data_dir,
    data_dir=data_dir,   
    file_name="df_master.pickle",
    process_df=True,
    )

df_m = df_ads

# ORR_Free_E_Plot Instance

In [5]:
ORR_PLT = ORR_Free_E_Plot(
    free_energy_df=None,
    state_title="adsorbate",
    free_e_title="ads_e",
    smart_format=smart_format_dict,

#     bias=0.,
    color_list=None,   
    show_H_e_pairs_annotations=True,
    show_legend=True,
    rxn_type="OER",
    )

# Processing Data

In [6]:
prop_name_list = [
    'bulk_system',
#     'coverage',
    'coverage_type',
    'facet',
    'surface_type',
    ]

grouped = df_m.groupby(groupby_props)

for i_ind, (name, group) in enumerate(grouped):
    df_i = group

    if not any([np.isnan(i) for i in df_i.elec_energy.tolist()]):        
        ORR_PLT.add_series(
            df_i,
            plot_mode="all",
            overpotential_type="OER",
            property_key_list=prop_name_list,
            add_overpot=False,
            )

In [7]:
scaling_dict_fitted = {
    
    "ooh": {
        "m": 1.,
        "b": 3.1,
        },
    "o": {
        "m": 2.,
        "b": 0,
        },
    "oh": {
        "m": 1.,
        "b": 0.,
        },
    }

scaling_dict_fitted = {
    
    "ooh": {
        "m": 0.976,
        "b": 3.09,
        },
    "o": {
        "m": 1.30,
        "b": 1.19,
        },
    "oh": {
        "m": 1.,
        "b": 0.,
        },
    }

# intercept_i:  3.0923565821584003
# slope_i:  0.9756829157533189

# Scaling fit for  o
# intercept_i:  1.3025046731746635
# slope_i:  1.1905482253235011

# Experimental IrOx Activity Traces

## Horizontal data traces

In [8]:
trace_iro3 = go.Scatter(
    x=plot_range["x"],
    y=2*[exp_irox_lim_pot["iro3"]["lim_pot"]],
    mode="lines",
    name="lines",
    line={
        "color": exp_irox_lim_pot["iro3"]["line_color"],
        "width": 1,
        "dash": "dash",
        },
    )

trace_iro2 = go.Scatter(
    x=plot_range["x"],
    y=2*[exp_irox_lim_pot["iro2"]["lim_pot"]],
    mode="lines",
    name="lines",
    line={
        "color": exp_irox_lim_pot["iro2"]["line_color"],
        "width": 1,
        "dash": "dash",
        },
    )

# trace_irox = go.Scatter(
#     x=plot_range["x"],
#     y=2*[exp_irox_lim_pot["irox"]["lim_pot"]],
#     mode="lines",
#     name="lines",
#     line={
#         "color": exp_irox_lim_pot["irox"]["line_color"],
#         "width": 1,
#         "dash": "dash",
#         },
#     )

## Exp. IrOx Annotations

In [9]:
annotations=[
    dict(
        x=1,
        y=exp_irox_lim_pot["iro3"]["lim_pot"],
        xref='x',
        yref='y',
        text='IrO<sub>3</sub> (@1mA/cm<sup>2</sup>)',
        showarrow=False,
        xanchor="left",
        yshift=9,
        ),

    dict(
        x=1,
        y=exp_irox_lim_pot["iro2"]["lim_pot"],
        xref='x',
        yref='y',
        text='IrO<sub>2</sub> (@1mA/cm<sup>2</sup>)',
        showarrow=False,
        xanchor="left",
        yshift=9,
        ),

#     dict(
#         x=1,
#         y=exp_irox_lim_pot["irox"]["lim_pot"],
#         xref='x',
#         yref='y',
#         text='IrO<sub>x</sub>',
#         showarrow=False,
#         xanchor="left",
#         yshift=7,
#         ),

    ]

# Volcano Plot

In [10]:
VP = Volcano_Plot(
    ORR_PLT,
    x_ax_species="o-oh",  # 'o-oh' or 'oh'
    smart_format_dict=smart_format_dict,
    plot_range=plot_range,
    )

VP.create_volcano_relations_plot()

volcano_legs_data = VP.create_volcano_lines(
    gas_molec_dict=gas_molec_dict,
    scaling_dict=scaling_dict_ideal,
    plot_all_legs=False,
    plot_min_max_legs=True,
    trace_priority="bottom",  # 'top' or 'bottom'
    )

volcano_legs_data_tmp = VP.create_volcano_lines(
    gas_molec_dict=gas_molec_dict,
    scaling_dict=scaling_dict_fitted,
    plot_all_legs=False,
    plot_min_max_legs=True,
    trace_priority="bottom",  # 'top' or 'bottom'
    legs_to_plot=[
#         "o2_to_ooh",
        "ooh_to_o",
        "o_to_oh",
#         "oh_to_h2o",
        ],
    line_color="grey"
    )

data = volcano_legs_data + volcano_legs_data_tmp + VP.data_points

if plot_exp_traces:
    data.insert(0, trace_iro3)
    data.insert(0, trace_iro2)
#     data.insert(0, trace_irox)

In [11]:
if save_plot:
    save_dir = proj_dir_name
else:
    save_dir = "__temp__"

In [12]:
layout = VP.get_plotly_layout(
    showlegend=True,
    width=3. * 9. * 37.795275591,
    height=4.5 * 9. * 37.795275591)
layout["annotations"] = annotations

fig = dict(data=data, layout=layout)
# py.iplot(fig, filename=os.path.join(save_dir, "pl_irox_volcano_plotly_large_low_energy_ooh"))
py.iplot(fig, filename=os.path.join(save_dir, "oer_volcano", "pl_irox_volcano_plotly_large_default_ooh"))

In [13]:
layout = VP.get_plotly_layout(
    showlegend=False,

    width=1.45 * 7.964 * 37.795275591,
#     width=0.3 * 7.964 * 37.795275591,

    height=1.5 * 5.6002 * 37.795275591)
layout["annotations"] = annotations
layout["xaxis"]["title"] = ""
layout["yaxis"]["title"] = ""

fig = dict(data=data, layout=layout)
py.iplot(fig, filename=os.path.join(save_dir, "oer_volcano", "pl_irox_volcano_plotly_default_ooh"))

In [14]:
layout["xaxis"]["title"] = ""

In [15]:
df_m[
    (df_m["bulk_system"] == "IrO3_rutile-like") &
    (df_m["adsorbate"] == "ooh") &
#     (df_m[""] == "") &
    [True for i in range(len(df_m))]
    ]

,bulk_system,facet,adsorbate,coverage_type,ooh_direction,ads_e,elec_energy,total_magmom,abs_magmom,path_short,...,incar_parsed,init_atoms,atoms_object,N_atoms,dipole_correction,path,name_i_2,name_i_3,priority,surface_type
259,IrO3_rutile-like,100,ooh,o_covered,up,4.757747,-552.246367,3.682,7.002,03_OER_Calc/IrO3_rutile-like/100/01_O-4_OH-0/0...,...,"{'ENCUT': 500.0, 'AMIX_MAG': 0.05, 'BMIX_MAG':...","(Atom('O', [1.84444, 2.83282, 7.50008], index=...","[(Atom('O', [1.84444, 2.83282, 7.50008], index...",90.0,True,/global/cscratch1/sd/flores12/IrOx_Project/03_...,100_O-4_OH-0_IrO3_rutile-like,"100, O-4_OH-0, IrO3_rutile-like,",NaN,NaN
265,IrO3_rutile-like,100,ooh,o_covered_2,up,4.205397,-543.880710,-0.576,8.490,03_OER_Calc/IrO3_rutile-like/100/02_O-2_OH-0/0...,...,"{'POTIM': '0.200000', 'ENCUT': 500.0, 'AMIX_MA...","(Atom('O', [1.84444, 2.83282, 7.50008], magmom...","[(Atom('O', [1.84444, 2.83282, 7.50008], index...",88.0,True,/global/cscratch1/sd/flores12/IrOx_Project/03_...,100_O-2_OH-0_IrO3_rutile-like,"100, O-2_OH-0, IrO3_rutile-like,",NaN,NaN
272,IrO3_rutile-like,100,ooh,h_covered,NaN,4.807029,-557.647162,3.683,6.729,03_OER_Calc/IrO3_rutile-like/100/03_O-2_OH-2/0...,...,"{'ENCUT': 500.0, 'AMIX_MAG': 0.05, 'BMIX_MAG':...","(Atom('O', [1.84444, 2.83282, 7.50008], index=...","[(Atom('O', [1.84444, 2.83282, 7.50008], index...",91.0,True,/global/cscratch1/sd/flores12/IrOx_Project/03_...,100_O-2_OH-2_IrO3_rutile-like,"100, O-2_OH-2, IrO3_rutile-like,",NaN,NaN
276,IrO3_rutile-like,110,ooh,o_covered,up,4.084265,-425.810928,0.853,2.781,03_OER_Calc/IrO3_rutile-like/110/01_O_covered/...,...,"{'ENCUT': 500.0, 'AMIX_MAG': 0.05, 'BMIX_MAG':...","(Atom('O', [3.32082, 2.83282, 7.50011], index=...","[(Atom('O', [3.32082, 2.83282, 7.50011], index...",68.0,True,/global/cscratch1/sd/flores12/IrOx_Project/03_...,110_o_covered_IrO3_rutile-like,"110, o_covered, IrO3_rutile-like,",NaN,NaN
282,IrO3_rutile-like,110,ooh,h_covered,NaN,4.120659,-431.021152,2.210,3.516,03_OER_Calc/IrO3_rutile-like/110/02_H_covered/...,...,"{'ENCUT': 500.0, 'AMIX_MAG': 0.05, 'BMIX_MAG':...","(Atom('O', [3.32082, 2.83282, 7.50011], index=...","[(Atom('O', [3.32082, 2.83282, 7.50011], index...",69.0,True,/global/cscratch1/sd/flores12/IrOx_Project/03_...,110_h_covered_IrO3_rutile-like,"110, h_covered, IrO3_rutile-like,",NaN,NaN


In [16]:
assert False

AssertionError: 

In [ ]:
from ase import io

In [ ]:
row_i = df_m[
    (df_m["bulk_system"] == "IrO3") &
    (df_m["coverage_type"] == "o_covered") &
    (df_m["facet"] == "100") &
    (df_m["adsorbate"] == "o") &
    [True for i in range(len(df_m))]
    ].iloc[0]

atoms_i = row_i["atoms_object"][-1]

io.write("iro3_o-covered_100_o.traj", atoms_i)

In [ ]:
row_i = df_m[
    (df_m["bulk_system"] == "IrO3") &
    (df_m["coverage_type"] == "o_covered") &
    (df_m["facet"] == "110") &
    (df_m["adsorbate"] == "o") &
    [True for i in range(len(df_m))]
    ].iloc[0]

atoms_i = row_i["atoms_object"][-1]

io.write("iro3_o-covered_110_o.traj", atoms_i)

In [ ]:
row_i = df_m[
    (df_m["bulk_system"] == "IrO3_rutile-like") &
    (df_m["coverage_type"] == "o_covered") &
    (df_m["facet"] == "110") &
    (df_m["adsorbate"] == "o") &
    [True for i in range(len(df_m))]
    ].iloc[0]

atoms_i = row_i["atoms_object"][-1]

io.write("iro3_rutile-like_o-covered_110_o.traj", atoms_i)

In [ ]:
row_i = df_m[
    (df_m["bulk_system"] == "IrO3_battery") &
    (df_m["coverage_type"] == "o_covered") &
    (df_m["facet"] == "010") &
    (df_m["adsorbate"] == "o") &
    [True for i in range(len(df_m))]
    ].iloc[0]

atoms_i = row_i["atoms_object"][-1]
atoms_i
io.write("iro3_rutile-like_o-covered_110_o.traj", atoms_i)

In [ ]:
df_i = df_m[
    (df_m["bulk_system"] == "IrO3_battery") &
    (df_m["coverage_type"] == "o_covered") &
    (df_m["facet"] == "010") &
    (df_m["adsorbate"] == "o") &
    (df_m["surface_type"] == "a") &
    [True for i in range(len(df_m))]
    ]

row_i = df_i.iloc[0]

atoms_i = row_i["atoms_object"][-1]

io.write("iro3-battery_o-covered_010_o_surface-type-a.traj", atoms_i)

In [ ]:
row_i = df_m[
    (df_m["bulk_system"] == "IrO3") &
    (df_m["coverage_type"] == "o_covered") &
    (df_m["facet"] == "211") &
    (df_m["adsorbate"] == "o") &
    [True for i in range(len(df_m))]
    ].iloc[0]

atoms_i = row_i["atoms_object"][-1]
atoms_i
io.write("iro3_o-covered_211_o.traj", atoms_i)

In [ ]:
row_i = df_m[
    (df_m["bulk_system"] == "IrO2") &
    (df_m["coverage_type"] == "o_covered") &
    (df_m["facet"] == "100") &
    (df_m["adsorbate"] == "o") &
    [True for i in range(len(df_m))]
    ].iloc[0]

atoms_i = row_i["atoms_object"][-1]
atoms_i
io.write("iro2_o-covered_100_o.traj", atoms_i)

In [ ]:
# df_dict = {}
#     df_dict["_".join(list(name))] = df_i
#     # Choosing the most stable *OOH species
#     # ###################################################
#     species_j = "ooh"
#     df_wo_species = df_i[df_i["adsorbate"] != species_j]
#     df_ij = df_i[df_i["adsorbate"] == species_j]
#     df_final = df_wo_species.append(
#         df_ij.loc[df_ij["ads_e"].idxmin()]
#         )
#     df_final
#     df_i = df_final
#     # ###################################################
#     sys_i = df_i.iloc[0]["bulk_system"] + "_" + df_i.iloc[0]["facet"]
#     color_i = system_color_map[sys_i]
#             color=color_i,
#             opt_name=df_i["name_i"].tolist()[0],

# df_m.loc[df_m["coverage_type"] == "O-4_OH-0", "coverage_type"] = "o_covered"
# df_m.loc[df_m["coverage_type"] == "O-2_OH-0", "coverage_type"] = "o_covered_2"
# df_m.loc[df_m["coverage_type"] == "O-2_OH-2", "coverage_type"] = "h_covered"

# df_m = df_m[df_m["ooh_direction"] != "deprotonated"]